In [1]:
import time
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# data from:
# https://www.imdb.com/interfaces/


# Load descriptions data

In [3]:
fpath = "./title.basics.tsv/data.tsv"
df_title = pd.read_csv(fpath, sep='\t')

years = [2000, 2001, 2002, 2003, 2004, 2005]

df = pd.DataFrame()
for year in years:
    df_aux = pd.read_csv('IMDB_movie_description_'+str(year)+'.csv')
    df = df.append(df_aux)
df = df.drop(['Unnamed: 0'], axis=1)
df = df.merge(df_title, on='tconst', how='left')
df = df[['tconst', 'description', 'primaryTitle', 'originalTitle',
       'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']]

C:\Users\CANCELLIERIE\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,tconst,description,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0113026,"The Fantasticks is a movie starring Joel Grey,...",The Fantasticks,The Fantasticks,0,2000,\N,86,"Musical,Romance"
1,tt0113092,"For the Cause is a movie starring Dean Cain, T...",For the Cause,For the Cause,0,2000,\N,100,"Action,Adventure,Drama"
2,tt0118694,Faa yeung nin wa is a movie starring Tony Chiu...,In the Mood for Love,Faa yeung nin wa,0,2000,\N,98,"Drama,Romance"
3,tt0118852,"Chinese Coffee is a movie starring Al Pacino, ...",Chinese Coffee,Chinese Coffee,0,2000,\N,99,Drama
4,tt0119273,Heavy Metal 2000 is a movie starring Michael I...,Heavy Metal 2000,Heavy Metal 2000,0,2000,\N,88,"Action,Adventure,Animation"


# Tfidf Vectorizer

In [145]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# list of text documents
text = df['description'].values

# create the transform
# tokenize and build vocab
stop_words = stopwords.words('english')
vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=250, lowercase=True)
#vectorizer = TfidfVectorizer(stop_words=stop_words, lowercase=True)
vector     = vectorizer.fit_transform(text)

# summarize
#print(vectorizer.vocabulary_)
print(vector.shape)
#print(type(vector))
#print(vector.toarray())

(6289, 2464)


C:\Users\CANCELLIERIE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


# KMeans clustering

In [146]:
from sklearn         import metrics
from sklearn.cluster import KMeans

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(vector)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [147]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [148]:
for i in range(true_k):
    print('Cluster %d:' % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

Cluster 0:
 year
 david
 de
 father
 years
 lives
 james
 group
 school
 documentary
Cluster 1:
 town
 small
 friend
 peter
 best
 time
 wife
 living
 village
 school
